In [ ]:
import pandas as pd

# Load the raw wifi data, convert it to pandas datetime format

In [ ]:
wifi_data = pd.read_csv('wifi.csv')
wifi_data['time'] = pd.to_datetime(wifi_data['time'])

# Remove duplicate records (this happens nearly exclusively a lot during the January - March period, unclear why)

In [ ]:
wifi_data = wifi_data.drop_duplicates(['time', 'Authenticated Client Count'])

# Iterate through all buildings in the wifi data, summing all readings for each timestmap per building (each building will have several readings for each timestamp)
# Take average connections in a 1 hour period using pd.grouper
# add building name to the df
# add the building-specific dataframe to the all_data df

In [ ]:
all_data = pd.DataFrame()
for building in wifi_data['Building'].unique():
    wifi_building_data = wifi_data[wifi_data['Building'] == building][['time', 'Authenticated Client Count']].groupby('time').sum().reset_index()
    wifi_building_data = wifi_building_data.groupby(pd.Grouper(key='time', freq='1H')).mean().reset_index()   
    wifi_building_data['building'] = building
    all_data = all_data.append(wifi_building_data)
    print(building)

# Load the Wifi building names and their associated building codes. This is used to jo

In [ ]:
wifi_buildings = pd.read_csv('wifi_buildings.csv')

In [ ]:
all_data = all_data.reset_index()

In [ ]:
wifi_data_withcodes = pd.merge(all_data, wifi_buildings, how='inner', on='building')

In [ ]:
wifi_data_withcodes.groupby('time').sum()['Authenticated Client Count'].mean()

In [ ]:
wifi_data_withcodes[['time', 'Authenticated Client Count', 'buildingCode']].to_csv('wifi_data_fin2.csv', header=True, date_format='%Y-%m-%dT%H:%M:%SZ')